## Edit distance

The code implements a recursive version of the edit distance between two strings x and y. It computes the minimum number of insertions, deletions, and substitutions needed to transform x into y. Note that this implementation is simple but inefficient for long strings because it recalculates subproblems repeatedly.

In [1]:
def editDistRecursive(x, y):
    # This implementation is very slow
    #if one string is empty, the distance is the length of the other
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1  # Delete last character of x (horizontal move)
        distVer = editDistRecursive(x, y[:-1]) + 1  # Delete last character of y (vertical move)
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])  #Substitute last character if different, or no cost if equal (diagonal move)
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)       # Return the minimum of the three possibilities

This function computes the edit distance between two strings x and y using dynamic programming. It calculates the minimum number of insertions, deletions, and substitutions required to transform x into y. Faster than the recursive version

In [3]:
def editDistance(x,y):
    # Initialize distance matrix D with size (len(x)+1) x (len(y)+1)
    D = []
    for i in range(len(x)+1):
        D.append([0]* (len(y)+1))
        
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1  # Cost of insertion (from left)
            distVer = D[i-1][j] + 1  # Cost of deletion (from top)
            if x[i-1] == y[j-1]:     # Cost of substitution (from diagonal)
                distDiag = D[i-1][j-1]   # no cost if characters match
            else:
                distDiag = D[i-1][j-1] + 1   # substitution cost
            D[i][j] = min(distHor, distVer, distDiag)   # Minimum of three possibilities
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [5]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistRecursive(x, y)

CPU times: total: 11.5 s
Wall time: 11.7 s


3

In [6]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistance(x, y)

CPU times: total: 0 ns
Wall time: 0 ns


3

## Global alignment

In [7]:
alphabet = ['A', 'C', 'G', 'T']
score = [[0, 4, 2, 4, 8],
         [4, 0, 4, 2, 8],
         [2, 4, 0, 4, 8],
         [4, 2, 4, 0, 8],
         [8, 8, 8, 8, 8]]

This function performs a global sequence alignment between two sequences x and y using dynamic programming. It computes the alignment score based on a given substitution matrix (score) and an alphabet allowing for mismatches and gaps. It returns the optimal global alignment score (minimized cost) for aligning the entire sequences.

In [8]:
def globalAlignment(x,y):
    # Initialize DP matrix with dimensions (len(x)+1) x (len(y)+1)
    D = []
    for i in range(len(x)+1):
        D.append([0]* (len(y)+1))
        
    for i in range(1, len(x)+1):  # Initialize first column (gap penalties for aligning x with empty y)
        D[i][0] = D[i-1][0] + score[alphabet.index(x[i-1])][-1]
    for i in range(1, len(y)+1):  # Initialize first row (gap penalties for aligning y with empty x)
        D[0][i] = D[0][i-1] + score[-1][alphabet.index(y[i-1])]
        
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]  # Cost of inserting a gap in x (move horizontally)
            distVer = D[i-1][j] + score[alphabet.index(x[i-1])][-1]  # Cost of inserting a gap in y (move vertically)
            if x[i-1] == y[j-1]:         # Cost of aligning characters (diagonal)
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]  # mismatch cost
                
            D[i][j] = min(distHor, distVer, distDiag) # Choose the minimum cost among horizontal, vertical, and diagonal moves
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [9]:
x = 'TATGTCATGC'
y = 'TATGGCAGC'
print(globalAlignment(x,y))

12


In [14]:
overlap('TTACGT', 'CGTACCGT')

3

In [15]:
overlap('TTACGT', 'GTACCGT')

0

## Overlaps 

This function finds the longest exact suffix-prefix overlap between two sequences a and b. It returns the length of the overlap that is at least min_length bases long. If no such overlap exists it returns 0. This is commonly used when constructing overlap graphs for genome assembly.

In [16]:
def overlap (a,b,min_length = 3):
    start = 0      # Start searching from the beginning of 'a'
    
    while True:
        # Find the first occurrence of b's prefix (length=min_length) in a, starting at 'start'
        start = a.find(b[:min_length], start)
        if start == -1:  # Exit if no occurrence found
            return 0
        if b.startswith(a[start]):   # Check if the suffix of 'a' starting at 'start' matches the prefix of 'b'
            return len(a)-start      # Length of the overlap
        start += 1   # Move past previous match to search again

In [17]:
overlap('TTACGT', 'CGTACCGT')

3

In [18]:
overlap('TTACGT', 'GTACCGT')

0

## Finding all overlaps 

In [19]:
from itertools import permutations

list(permutations([1,2,3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

This function implements a naive approach to compute all suffix-prefix overlaps of length at least k between a set of reads. It examines every pair of reads and stores the overlap length in a dictionary.

In [20]:
def naive_overlap_map(reads, k):
    olaps = {}    # Dictionary to store overlaps
    # Iterate over all ordered pairs of reads
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)   # Compute overlap length
        if olen > 0:    # Only keep overlaps of length >= k
            olaps[(a, b)] = olen
    return olaps

In [21]:
reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}
